In [135]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import torch
import torchtext
import jieba
import re
import pandas as pd
from tqdm.notebook import tqdm

# 1.训练Glove中文词向量

### 1.1将文本分词并去除停用词后按行写入文件

In [143]:
df = pd.read_csv('../data/THUCNews/train.csv').dropna().sample(100000)
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]

f = open('./stanford-Glove/THUCNews.txt', 'w')
for title in tqdm(df['title']):   
    # 去除标点符号
    title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
    tokens = [token for token in jieba.cut(title.strip()) if token not in stopwords]
    f.write(' '.join(tokens) + '\n')
f.close()

  0%|          | 0/100000 [00:00<?, ?it/s]

### 1.2使用standford/Glove工具训练

In [137]:

# cd stanford-Glove
# make
# sh demo.sh

# 2.加载Glove词向量

In [138]:
# 加载Glove预训练的词向量
embeddings = torchtext.vocab.Vectors(name ='chinese-glove-vectors.txt')

In [139]:
# 简单计算几个词之间的余弦相似度
def cal_cos(token1, token2):
    cos = torch.nn.CosineSimilarity(dim=0)
    return cos(embeddings.get_vecs_by_tokens(token1), embeddings.get_vecs_by_tokens(token2)).item()

print(cal_cos('中国', '美国'))
print(cal_cos('中国', '韩国'))
print(cal_cos('中国', '第一'))
print(cal_cos('图', '组图'))
print(cal_cos('巴萨', '皇马'))
print(cal_cos('巴萨', '中国'))

0.5108680129051208
0.42951643466949463
0.3974078893661499
0.26060956716537476
0.6339325308799744
0.22046658396720886


In [142]:
# 将预训练vectors加载到Embdding网络中
# freeze为True，则冻结embed层的参数
embed = torch.nn.Embedding.from_pretrained(embeddings.vectors, freeze=True)  
print(embed.weight)
print(embed.weight.requires_grad)

Parameter containing:
tensor([[ 7.9292e-01,  4.0965e-01,  3.9356e-01,  ...,  6.7521e-01,
          4.5740e-01, -1.5034e-01],
        [-1.2385e-01,  5.2830e-01, -1.1919e+00,  ...,  3.7324e-01,
          1.4801e+00,  8.9940e-03],
        [-1.7443e-01,  7.7462e-01,  8.4921e-01,  ...,  2.7134e-01,
          3.6000e-04,  3.0040e-01],
        ...,
        [ 2.7707e-01,  1.8921e-01,  1.3570e-01,  ..., -7.4619e-02,
          2.4220e-01,  8.8506e-02],
        [ 1.0361e-01,  8.5422e-02, -5.5904e-02,  ...,  4.0705e-01,
         -1.1426e-01, -1.8612e-01],
        [-1.2353e-02,  8.4600e-04, -7.1280e-03,  ..., -6.1163e-02,
         -6.0530e-03,  2.0306e-02]])
False
